In [ ]:
#import matplotlib
#matplotlib.use("PDF")
from pylab import *
import sys, time


In [ ]:
# A simple toy model with two pixels and an edge which moves as a function of the difference between the counts

def Edge(alpha,N1,N2):
    return 0.5 - alpha * (N1 - N2)

In [ ]:
import numpy as np
electrons_per_pixel = 10000
NumTrials = 1000
alphas = [1.0E-5]#[0.0,1.0E-7,2.0E-7,4.0E-7,8.0E-7,1.0E-6,2.0E-6,4.0E-6,8.0E-6,1.0E-5]
NumAlphas = len(alphas)
edge = zeros([3*electrons_per_pixel, NumTrials])+0.5
m0 = zeros([NumAlphas])
m1 = zeros([NumAlphas])
var0 = zeros([NumAlphas])
var1 = zeros([NumAlphas])
covar = zeros([NumAlphas])
for i,alpha in enumerate(alphas):
    N = zeros([2,NumTrials],dtype = int)
    for trial in range(NumTrials):
        electrons = np.random.poisson(lam = 2*electrons_per_pixel,size=1)
        #print electrons
        for j in range(electrons[0]):
            this_edge = Edge(alpha,N[0,trial],N[1,trial])
            edge[j,trial] = this_edge
            if rand() > this_edge:
                N[1,trial] += 1
            else:
                N[0,trial] += 1
        #print "N0 = %d, N1 = %d"%(N[0,trial],N[1,trial])

    m0[i] = mean(N[0,:])
    m1[i] = mean(N[1,:])
    print "M0 = %f, M1 = %f"%(m0[i],m1[i])
    for j in range(NumTrials):
        var0[i] += (N[0,j]-m0[i]) * (N[0,j]-m0[i])
        var1[i] += (N[1,j]-m1[i]) * (N[1,j]-m1[i])
        covar[i] += (N[0,j]-m0[i]) * (N[1,j]-m1[i])
    var0[i] /= float(NumTrials)
    var1[i] /= float(NumTrials)
    covar[i] /= float(NumTrials)
    covar[i] /= var0[i]
    print "Var0 = %f, Var1 = %f, Alpha = %f, Covar = %f"%(var0[i],var1[i],alpha,covar[i])

        
        

In [ ]:
mean_movement = zeros([3*electrons_per_pixel])

for i in range(NumTrials):
    for j in range(3*electrons_per_pixel):
        if abs(0.5 - edge[3*electrons_per_pixel - 1 - j, i]) < 1.0E-8:
            continue
        else:
            last_edge = (edge[3*electrons_per_pixel - 1 - j, i] - 0.50)
            last_n = 3*electrons_per_pixel - 1 - j
            break
    #print i, last_edge, last_n
    for j in range(last_n):
        mean_movement[j] += (edge[j, i] - 0.50) / (last_edge * float(last_n))

last_n = 18000
plot(mean_movement[0:last_n])
ave = mean(mean_movement[0:last_n]) / mean_movement[last_n]
text(100, 0.04, "Average/Final = %.3f"%ave)
#legend()
show()

In [ ]:
print mean(mean_movement[0:15000]) / mean_movement[15000]

In [ ]:
for i in range(NumTrials)[0:10]:
    plot(edge[:,i], label = "%d, %d"%(N[0,i],N[1,i]))
legend()
show()

In [ ]:

figure()
suptitle('Toy Model of B-F Covariance')
subplots_adjust(hspace = 0.5)
subplot(2,1,1)
plot(alphas,covar)
xscale('log')
yscale('log')
xlabel('Alpha')
ylabel('Covariance')
subplot(2,1,2)
plot(alphas,var0/10000.0)
xscale('log')
yscale('log')
ylim(0.5,1.0)
xlabel('Alpha')
ylabel('Variance / Mean')
show()

In [ ]:
from scipy import stats
file = open("/Users/cslage/Research/LSST/code/covariance/darts/toy/Toy_Covariances_50K_100K.txt")
lines = file.readlines()
file.close
areas = []
alphas = []
var = []
covar = []
for line in lines:
    items = line.split()
    if items[0] == "Alpha*1E6":
        continue

    alphas.append(float(items[0])/1.0E6)
    areas.append(float(items[0])/1.0E6*50000)
    var.append((float(items[3]) + float(items[4])) / 2.0)
    covar.append(float(items[5]))
alphas = array(alphas)
areas = array(areas)
var = array(var)
covar = array(covar)
figure()
subplots_adjust(hspace = 0.5)
suptitle('Toy Model of B-F Covariance')
subplot(2,1,1)
slope, intercept, r_value, p_value, std_err = stats.linregress(areas,covar/var)

xplot = linspace(0.0,0.1, 100)
yplot = intercept + slope * xplot
slope2 = 1.0
yplot2 = slope2 * xplot

plot(areas,covar/var, "ro", label = "Covariance")
plot(xplot, yplot, ls = "--", color = 'green', label = "Best Fit, slope = %.2f"%slope)
plot(xplot, yplot2, ls = "--", color = 'blue', label = "Theory, slope = %.2f"%slope2)

xlim(0,0.1)
ylim(0,0.1)
xlabel('$\delta A / A (N1 = 50K, N2 = 0)$')
ylabel('Covariance')
legend(loc = 'lower right', fontsize = 12)     
text(0.01, 0.07, "50K e-/pixel, 100K Trials")

subplot(2,1,2)
slope, intercept, r_value, p_value, std_err = stats.linregress(alphas,covar/var)

xplot = linspace(0.0,2.0E-6, 100)
yplot = intercept + slope * xplot
slope2 = 50000.0
yplot2 = slope2 * xplot
plot(alphas*1.0E6,covar/var, "ro", label = "Covariance")
plot(xplot*1.0E6, yplot, ls = "--", color = 'green', label = "Best Fit, slope = %.2f"%slope)
plot(xplot*1.0E6, yplot2, ls = "--", color = 'blue', label = "Theory, slope = %.2f"%slope2)

xlim(0,2.0)
ylim(0,0.1)
xlabel('Alpha*1E6')
ylabel('Covariance')
legend(loc = 'lower right', fontsize = 12)     
text(0.2, 0.07, "50K e-/pixel, 100K Trials")

savefig("/Users/cslage/Research/LSST/code/covariance/darts/toy/Covariance_50K_100K.pdf")   




In [ ]:
from scipy import stats
file = open("/Users/cslage/Research/LSST/code/covariance/darts/toy/Toy_Covariances_Pairs_10K_100K.txt")
lines = file.readlines()
file.close
areas = []
alphas = []
var = []
covar = []
for line in lines:
    items = line.split()
    if items[0] == "Alpha*1E6":
        continue

    alphas.append(float(items[0])/1.0E6)
    areas.append(float(items[0])/1.0E6*10000)
    var.append((float(items[3]) + float(items[4])) / 2.0)
    covar.append(float(items[5]))
alphas = array(alphas)
areas = array(areas)
var = array(var)
covar = array(covar)
figure()
subplots_adjust(hspace = 0.5)
suptitle('Toy Model of B-F Covariance - Difference of Pairs')
subplot(2,1,1)
slope, intercept, r_value, p_value, std_err = stats.linregress(areas,covar/var)

xplot = linspace(0.0,0.1, 100)
yplot = intercept + slope * xplot
slope2 = 1.0
yplot2 = slope2 * xplot

plot(areas,covar/var, "ro", label = "Covariance")
plot(xplot, yplot, ls = "--", color = 'green', label = "Best Fit, slope = %.2f"%slope)
plot(xplot, yplot2, ls = "--", color = 'blue', label = "Theory, slope = %.2f"%slope2)

xlim(0,0.1)
ylim(0,0.1)
xlabel('$\delta A / A$')
ylabel('Covariance')
legend(loc = 'lower right', fontsize = 12)     
text(0.01, 0.07, "10K e-/pixel, 100K Trials")

subplot(2,1,2)
slope, intercept, r_value, p_value, std_err = stats.linregress(alphas,covar/var)

xplot = linspace(0.0,1.0E-3, 100)
yplot = intercept + slope * xplot
slope2 = 10000.0
yplot2 = slope2 * xplot
plot(alphas*1.0E6,covar/var, "ro", label = "Covariance")
plot(xplot*1.0E6, yplot, ls = "--", color = 'green', label = "Best Fit, slope = %.2f"%slope)
plot(xplot*1.0E6, yplot2, ls = "--", color = 'blue', label = "Theory, slope = %.2f"%slope2)

xlim(0,10.0)
ylim(0,0.1)
xlabel('Alpha*1E6')
ylabel('Covariance')
legend(loc = 'lower right', fontsize = 12)     
text(1.0, 0.07, "10K e-/pixel, 100K Trials")

savefig("/Users/cslage/Research/LSST/code/covariance/darts/toy/Covariance_Pairs_10K_100K.pdf")   


